In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Mai 30

@author: yaning
"""

import numpy as np
import tensorflow as tf
# from tensorflow.keras import layers, models, datasets
import matplotlib.pyplot as plt
np.set_printoptions(threshold=np.inf)
import pickle
path = "/home/yaning/Documents/"

2025-10-20 15:34:38.721829: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 15:34:38.979463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 15:34:40.164625: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
with open(path + "gabor_data_mnst.pkl", "rb") as f:
    normalised_gabor_data = pickle.load(f)

In [3]:
normalised_gabor_data.shape

(30, 10, 10, 8)

In [4]:
# form an shape (5000, 3) array with 5000 time points and 3 input neurons
# every fire is 45-50 cycles, 
pointCount = 6000
start_tstep = 500
one_cycle = np.concatenate((np.ones(45, dtype=bool), np.zeros(189, dtype=bool)))
# one_cycle = np.ones(234)
firing_duration = 45+189
# 23*(45+189) = 5382
total_time = 5382


non_fire_array = np.zeros(pointCount, dtype=bool)

fire_seq = np.tile(one_cycle, 23)
fire_array = non_fire_array.copy()
fire_array[start_tstep:start_tstep+len(fire_seq)] = fire_seq

In [5]:
normalised_gabor_data.shape

(30, 10, 10, 8)

In [6]:
fire_data = np.zeros((30,10,10,8, 4000), dtype=bool)
for i in range(normalised_gabor_data.shape[0]):
    for x in range(normalised_gabor_data.shape[1]):
        for y in range(normalised_gabor_data.shape[2]):
            for z in range(normalised_gabor_data.shape[3]):
                if normalised_gabor_data[i,x,y,z] == 0:
                    fire_data[i,x,y,z] = non_fire_array[:4000]
                else:
                    fire_data[i,x,y,z] = fire_array[:4000]

In [16]:
fire_data.shape

(30, 10, 10, 8, 4000)

In [8]:
with open("datasets/mnst_30.pkl", "rb") as f:
    imgs = pickle.load(f)

In [23]:
imgs["tag"][17]

np.uint8(8)

In [21]:
for i in range(imgs["tag"].shape[0]):
    if imgs["tag"][i] == 9:
        print(i)

4
19
22


In [14]:
chosen_fire_data = fire_data[19]

In [25]:
chosen_fire_data = np.concatenate([chosen_fire_data] * 4, axis=-1)

In [34]:
chosen_fire_data.shape

(10, 10, 8, 4000)

In [35]:
p = 0.01
mask = np.random.rand(*chosen_fire_data.shape) < p
b_chosen_fire_data = chosen_fire_data | mask

In [17]:
with open(path + "fire_data_mnst_all.pkl", "wb") as f:
    pickle.dump(fire_data, f)

In [15]:
with open(path + "fire_data_mnst_nine_another.pkl", "wb") as f:
    pickle.dump(chosen_fire_data, f)